In [18]:
from docplex.mp.model import Model
import pandas

In [19]:
df_costo_cliente_planta = pandas.read_excel("./data.xlsx", sheet_name="COSTO CLIENTE-PLANTA")
df_clientes = pandas.read_excel("./data.xlsx", sheet_name="DEMANDAS CLIENTES")
df_plantas = pandas.read_excel("./data.xlsx", sheet_name="COSTO Y CAPACIDAD PLANTAS")

**Conjuntos**

In [20]:
PLANTAS = df_plantas["Planta"].to_list()
CLIENTES = df_clientes["Cliente"].to_list()

**Parámetros**

In [21]:
COSTO_CLIENTE_PLANTA = df_costo_cliente_planta.set_index(["Planta", "Cliente"])['Costo de asignar un cliente a un planta'].to_dict()
DEMANDA_CLIENTE = df_clientes.set_index(["Cliente"])["Demanda del cliente"].to_dict()
COSTO_ABRIR_PLANTA = df_plantas.set_index(["Planta"])["Costo de abrir planta"].to_dict()
CAPACIDAD_PLANTA = df_plantas.set_index(["Planta"])["Capacidad máxima"].to_dict()

In [22]:
model = Model()

**Variables**

In [23]:
x = model.binary_var_dict(PLANTAS, name="x")
y = model.binary_var_dict([(planta, cliente) for planta in PLANTAS for cliente in CLIENTES], name="y")

alpha = model.binary_var_dict(PLANTAS, name="alpha")

capacidad_utilizada = model.continuous_var_dict(PLANTAS, name="capacidad_utilizada")
beta = model.binary_var_dict(PLANTAS, name="beta")
pago095 = model.continuous_var_dict(PLANTAS, name="p", lb=-1000)

**Función Objetivo**

In [24]:
costo_abrir_plantas = model.sum(x[planta] * COSTO_ABRIR_PLANTA[planta] for planta in PLANTAS)
costo_cliente_planta = model.sum(y[(planta, cliente)] * COSTO_CLIENTE_PLANTA[(planta, cliente)] for planta in PLANTAS for cliente in CLIENTES)
costo_095 = model.sum(pago095[planta] for planta in PLANTAS)
model.minimize(costo_abrir_plantas + costo_cliente_planta + costo_095)

**Restricciones**

In [25]:
for planta in PLANTAS:
    for cliente in CLIENTES:
        model.add_constraint(y[(planta, cliente)] <= x[planta], ctname=f"ct1_{planta}_{cliente}")

for planta in PLANTAS:
    model.add_constraint(model.sum(y[(planta, cliente)] * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES) <= CAPACIDAD_PLANTA[planta], ctname=f"ct2_{planta}")

for cliente in CLIENTES:
    model.add_constraint(model.sum(y[(planta, cliente)] for planta in PLANTAS) == 1, ctname=f"ct3_{cliente}")

for planta in PLANTAS:
    model.add_constraint(model.sum(y[(planta, cliente)] for cliente in CLIENTES) >= 3 * x[planta], ctname=f"ct4_{planta}")

for planta in PLANTAS:
    model.add_constraint(model.sum(y[(planta, cliente)] * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES) >= 0.9 * CAPACIDAD_PLANTA[planta] * x[planta], ctname=f"ct5_{planta}")

$$ \forall p\in P: \text{CAPAC}_{p} \times CU_{p} = \sum_{c\in C} Y_{pc} \times D_{c} $$

In [26]:
for planta in PLANTAS:
    model.add_constraint(CAPACIDAD_PLANTA[planta] * capacidad_utilizada[planta] == model.sum(y[(planta, cliente)] * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES))

$$ \forall p\in P: \mathcal{M} \times \beta_{p} \le \mathcal{M} + CU_{p} - 0.95 $$
$$ \forall p\in P: \mathcal{M} \times \beta_{p} \ge CU_{p} - 0.95 $$

In [27]:
for planta in PLANTAS:
    model.add_constraint(10 * beta[planta] <= 10 + (capacidad_utilizada[planta] - 0.95))
    model.add_constraint(10 * beta[planta] >= (capacidad_utilizada[planta] - 0.95))

$$ \forall p\in P: X_{pc} = 0 \to \text{Pago095}_{p}=0 $$
$$ \forall p\in P: X_{pc} = 1 \to \text{Pago095}_{p}=-500\times\beta_{p}+500\times(1-\beta_{p}) $$

In [28]:
# for planta in PLANTAS:
#     model.add_indicator(x[planta], pago095[planta] == 0, active_value=0)
#     model.add_indicator(x[planta], pago095[planta] == -500 * (beta[planta]) + 500 * (1 - beta[planta]), active_value=1)

$$ \text{Pago095}_{p} \le -500\times\beta_{p}+500\times(1-\beta_{p})+\mathcal{M}\times (1-X_{p}) $$
$$ \text{Pago095}_{p} \ge -500\times\beta_{p}+500\times(1-\beta_{p})-\mathcal{M}\times (1-X_{p}) $$
$$ \text{Pago095}_{p} \le 0+\mathcal{M}\times X_{p} $$
$$ \text{Pago095}_{p} \ge 0-\mathcal{M}\times X_{p} $$

In [29]:
for planta in PLANTAS:
    model.add_constraint(pago095[planta] <= -500 * (beta[planta]) + 500 * (1 - beta[planta]) + 10000 * (1 - x[planta]))
    model.add_constraint(pago095[planta] >= -500 * (beta[planta]) + 500 * (1 - beta[planta]) - 10000 * (1 - x[planta]))
    model.add_constraint(pago095[planta] <= 0 + 10000 * (x[planta]))
    model.add_constraint(pago095[planta] >= 0 - 10000 * (x[planta]))

In [30]:
model.set_time_limit(20)

In [31]:
solution = model.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               20
Tried aggregator 1 time.
MIP Presolve eliminated 74 rows and 94 columns.
MIP Presolve modified 1171 coefficients.
Reduced MIP has 1176 rows, 1006 columns, and 6822 nonzeros.
Reduced MIP has 966 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (9.69 ticks)
Probing fixed 0 vars, tightened 1 bounds.
Probing time = 0.00 sec. (3.33 ticks)
Cover probing fixed 0 vars, tightened 3 bounds.
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 13 coefficients.
Reduced MIP has 1176 rows, 1006 columns, and 6822 nonzeros.
Reduced MIP has 966 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (5.49 ticks)
Probing time = 0.00 sec. (2.24 ticks)
Clique table members: 1134.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministi

In [32]:
solution

docplex.mp.solution.SolveSolution(obj=33767,values={x_3:1,x_4:1,x_5:1,x_..

In [33]:
model.get_solve_status()

<JobSolveStatus.FEASIBLE_SOLUTION: 1>

In [34]:
model.objective_value

33766.99999989415

In [35]:
df = pandas.DataFrame(columns=["X", "CU", "β", "Pago_095"])
for planta in PLANTAS:
    df.loc[planta, "X"] = x[planta].solution_value
    df.loc[planta, "CU"] = capacidad_utilizada[planta].solution_value
    df.loc[planta, "β"] = beta[planta].solution_value
    df.loc[planta, "Pago_095"] = pago095[planta].solution_value
df

,X,CU,β,Pago_095
1,0,0,0,0
2,0,0,0,0
3,1.0,1.0,1.0,-500.0
4,1.0,0.961538,1.0,-500.0
5,1.0,1.0,1.0,-500.0
6,1.0,0.957895,1.0,-500.0
7,1.0,0.965517,1.0,-500.0
8,0,0,0,0
9,1.0,0.95,1.0,-500.0
10,1.0,0.969072,1.0,-500.0


In [36]:
for planta in PLANTAS:
    if x[planta].solution_value > 0.99:
        print(f"=== Planta {planta} ====")
        print(f"Capacidad Planta: {CAPACIDAD_PLANTA[planta]}")
        print(f"Cantidad Enviada: {sum(y[(planta, cliente)].solution_value * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES)}")
        print(f"Capacidad Utilizada: {capacidad_utilizada[planta].solution_value}")
        print(f"Pago 0.95: {pago095[planta].solution_value}")
        for cliente in CLIENTES:
            if y[(planta, cliente)].solution_value > 0.99:
                print(cliente, end="  ")
        print("\n")

=== Planta 3 ====
Capacidad Planta: 32
Cantidad Enviada: 32.0
Capacidad Utilizada: 1.0
Pago 0.95: -500.0
10  38  49  

=== Planta 4 ====
Capacidad Planta: 52
Cantidad Enviada: 50.0
Capacidad Utilizada: 0.9615384615384616
Pago 0.95: -500.0
12  17  24  

=== Planta 5 ====
Capacidad Planta: 32
Cantidad Enviada: 32.0
Capacidad Utilizada: 1.0
Pago 0.95: -500.0
11  16  37  

=== Planta 6 ====
Capacidad Planta: 95
Cantidad Enviada: 91.0
Capacidad Utilizada: 0.9578947368421052
Pago 0.95: -500.0
3  6  50  

=== Planta 7 ====
Capacidad Planta: 29
Cantidad Enviada: 28.000000055714285
Capacidad Utilizada: 0.9655172433004926
Pago 0.95: -500.0
7  35  36  

=== Planta 9 ====
Capacidad Planta: 80
Cantidad Enviada: 76.0
Capacidad Utilizada: 0.9499999999999995
Pago 0.95: -500.0
8  14  34  

=== Planta 10 ====
Capacidad Planta: 97
Cantidad Enviada: 94.0
Capacidad Utilizada: 0.9690721649484536
Pago 0.95: -500.0
4  25  47  

=== Planta 11 ====
Capacidad Planta: 23
Cantidad Enviada: 22.0
Capacidad Utilizada